In [2]:
#Packages
import pandas as pd
from pandas_datareader import data
import datetime as dt
from datetime import timedelta, date
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
#Variables
dateRange = 120
slopeThresh = .1
varThresh = .05

#(yes/no) Calculate Slope and Var thresh?
slopeVarCalc = 'yes'

#make sure to enter your path
excelOutputFolder = r'C:\Users\adamd\Desktop\Hobbies and Things\PandasOutputExtracts\\'

In [ ]:
#Create Clean Reset DF
statsDFBlank = pd.DataFrame(columns = ['Symbol', 'LastPrice', 'Days', 'StDev', 'Avg', 'Slope', 'Std/Avg'])
statsDF = statsDFBlank

#set date range
endDate = date.today()
dateRange = dt.timedelta(dateRange)
startDate = endDate - dateRange

#Pull all S&P stocks
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
w = wiki[0]
w = w['Symbol']
w = w.reset_index()
iterations = len(w)

#Generate Export DF
for x in range(iterations): 
    try:
        #Pick a ticker
        myTicker = w['Symbol'].iloc[x]

        #List Prices
        Prices = data.DataReader([myTicker], 'yahoo', start=startDate, end=endDate)         
        Prices = Prices['Adj Close']
        Prices = Prices.reset_index()

        #Find Stats
        stdPrice = Prices.std()[myTicker]
        avgPrice = Prices.mean()[myTicker]
        tradeDays = Prices.count()[myTicker]

        #Find slope
        slope, intercept, r, p, se = stats.linregress(Prices.index, Prices[myTicker])

        #Select most recent price
        mostrecent = Prices[myTicker].iloc[len(Prices)-1]

        #How volitaile
        percentStd = stdPrice / avgPrice

        #Add Export Data
        exData = {'Symbol': myTicker,
            'LastPrice': mostrecent,
            'Days': tradeDays,
            'StDev': stdPrice,
            'Avg': avgPrice,
            'Slope': slope,
            'Std/Avg': percentStd}
        statsDF = statsDF.append(exData, ignore_index = True)
    except:
        continue


In [ ]:
#view output
statsDF

In [ ]:
statsStatsDF = statsDF.describe()
statsStatsDF

In [ ]:
#function to set auto-limits
if slopeVarCalc == 'yes':
    varThresh = statsStatsDF['Std/Avg'].iloc[6]
    slopeThresh = statsStatsDF['Slope'].iloc[5]
    

In [ ]:
#apply Buy and Sell Price and Logic
statsDFTwo = statsDF
statsDFTwo['buyPrice'] = statsDFTwo['Avg'] - statsDFTwo['StDev']
statsDFTwo['sellPrice'] = statsDFTwo['Avg'] + statsDFTwo['StDev']
statsDFTwo.loc[statsDFTwo['LastPrice'] <= statsDFTwo['buyPrice'], 'Buy?'] = 'yes' 
statsDFTwo.loc[statsDFTwo['LastPrice'] >= statsDFTwo['sellPrice'], 'Sell?'] = 'yes'

statsDFTwo=statsDFTwo.fillna('no')

def f(row):
    if row['Buy?'] == 'yes' or row['Sell?'] == 'yes':
        val = 'yes'
    else:
        val = 'no'
    return val

statsDFTwo['Buy or Sell?'] = statsDFTwo.apply(f, axis=1)

def s(row):
    if row['Slope?'].abs() < slopeThresh and row['Std/Avg'] > varThresh:
        val = 'yes'
    else:
        val = 'no'
    return val
statsDFTwo['Qualifying Stock?'] = statsDFTwo.apply(f, axis=1)

statsDFTwo

In [ ]:
#filter  for only reccomended buys and sells


statsDFThree = statsDFTwo[statsDFTwo['Slope'].abs() < slopeThresh]  
statsDFThree = statsDFThree[statsDFThree['Std/Avg'] > varThresh]
statsDFThree = statsDFThree[statsDFThree['Buy or Sell?'] == 'yes']
statsDFThree

In [423]:
#Export all data to excel
statsDF.to_excel(excelOutputFolder+"StockStatsExport.xlsx")
statsDFTwo.to_excel(excelOutputFolder+"StockStatsExportwithCalcs.xlsx")
statsDFThree.to_excel(excelOutputFolder+"StockStatsExportwithCalcsRefined.xlsx")